From 21.06, we are accepting epmc evidence without pmids as well, where the only identifier is the PubmedCentral identifier. In such case, the literature field will be empty. But have to test.... so far we don't have any dataset where this value is missing.

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# establish spark connection
spark = (
    SparkSession.builder
    .master('local[*]')
    .getOrCreate()
)

cooccfile = '/Users/dsuveges/project_data/epmc_evidence/cooccurrences/part-00000-e5a8d013-8805-4403-8500-4483f2abaee0-c000.snappy.parquet'

epmc_df = (
    spark.read.parquet(cooccfile)
    .filter(
        (F.col('type') == 'GP-DS') &
        F.col('isMapped')
    )
)

(
    epmc_df
    .groupby('pmid')
    .agg(
        F.count('pmid').alias('pmid_count'),
        F.first('pmcid').alias('pmcid')
    )
    .orderBy('pmid_count', ascending=False)
    .show()
)

In [ ]:
pmid_to_remove = '11956232' # This pubmed ID will be removed to do a test run with the updated parser


# from pyspark.sql.functions import UserDefinedFunction

(
    epmc_df
    .withColumn('pmid_updated',
         F.when(F.col('pmid') == pmid_to_remove, None)
        .otherwise(F.col('pmid'))
    )
    .drop(F.col('pmid'))
    .withColumnRenamed('pmid_updated', 'pmid')
    .write.format('parquet').mode('overwrite')
    .save('/Users/dsuveges/project_data/epmc_evidence/cooccurrences/pmid_removed')
)



In [ ]:
(
    epmc_df
    .filter(F.col('pmcid') == 'PMC2199249')
    .show()
)

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# establish spark connection
spark = (
    SparkSession.builder
    .getOrCreate()
)

cooccfile = '/Users/dsuveges/project_data/epmc_evidence/cooccurrences/part-00000-e5a8d013-8805-4403-8500-4483f2abaee0-c000.snappy.parquet'


epmc_df = (
    spark.read.parquet(cooccfile)
    .filter(
        (F.col('type') == 'GP-DS') &
        F.col('isMapped')
    )
    .persist()
)

epmc_df.printSchema()

root
 |-- pmid: string (nullable = true)
 |-- pmcid: string (nullable = true)
 |-- pubDate: string (nullable = true)
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- organisms: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- section: string (nullable = true)
 |-- text: string (nullable = true)
 |-- association: struct (nullable = true)
 |    |-- Altered_Expression: double (nullable = true)
 |    |-- Any: double (nullable = true)
 |    |-- Genetic_Variation: double (nullable = true)
 |    |-- Negative: double (nullable = true)
 |    |-- Neutral: double (nullable = true)
 |    |-- No: double (nullable = true)
 |    |-- Positive: double (nullable = true)
 |    |-- Regulatory_modification: double (nullable = true)
 |    |-- Yes: double (nullable = true)
 |-- end1: long (nullable = true)
 |-- end2: long (nullable = true)
 |-- evidence_score: double (nullable = tr